In [1]:
import json
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# ISTANBUL ÇALIŞMASI

In [2]:
istanbul = pd.read_csv(r"C:\Users\emrecan.ulu.DESKTOP-4T87DPH\Desktop\türkiye\türkiye_monthly.csv")
istanbul = istanbul.drop(columns=("index"))
istanbul = istanbul.drop(columns=("Unnamed: 0"))
istanbul = istanbul.loc[istanbul.city == "İstanbul"]
istanbul.reset_index(drop=True, inplace=True)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\emrecan.ulu.DESKTOP-4T87DPH\\Desktop\\türkiye\\türkiye_monthly.csv'

In [ ]:
istanbul

In [ ]:
istanbul.dropna(thresh=1, inplace=True)
istanbul

In [ ]:
#boş avgları dolduruyorum
istanbul.loc[pd.isna(istanbul["tavg"]), "tavg"] = (istanbul["tmax"] + istanbul["tmin"]) / 2

In [ ]:
istanbul

In [ ]:
# Tarih sütununu datetime formatına çeviriyoruz
istanbul['time'] = pd.to_datetime(istanbul['time'])

# yıllık ortalama sıcaklıkları çekiyorum
istanbul_ortalama = istanbul.copy()
istanbul_ortalama.set_index("time", inplace=True)

# Yıllık ortalama sıcaklık hesaplaması
istanbul_ortalama = istanbul_ortalama.resample("Y").mean()

# Yıllık ortalama sıcaklık verilerini ayrı bir DataFrame olarak kaydetme
istanbul_ortalama = pd.DataFrame(istanbul_ortalama, columns=["tavg"])
istanbul_ortalama.reset_index(drop=False, inplace=True)

# 10 yıllık hareketli ortalamaları al
ten_year_avg = istanbul_ortalama["tavg"].rolling(window=10).mean()

# Grafiğin boyutunu ayarlayın
plt.figure(figsize=(12, 8))

# Arka planı beyaz yerine grid tarzı yapın
sns.set(style="whitegrid")

# Grafikleri çizdirme
plt.plot(istanbul_ortalama["time"], istanbul_ortalama["tavg"], label="Yıllık Ortalama Sıcaklık", color="b")
plt.plot(istanbul_ortalama["time"], ten_year_avg, label="10 Yıllık Ortalama Sıcaklık", linewidth=2, color="r", marker='o')

# Eksen etiketlerini ve başlığı güncelleme
plt.xlabel("Tarih")
plt.ylabel("Sıcaklık Ortalaması (°C)")
plt.title("İstanbul Yıllık Ortalama Sıcaklıklar ve 10 Yıllık Ortalama")

# İlgili yıla odaklanmak için grafiği sınırlama
plt.xlim(istanbul_ortalama["time"].iloc[0], istanbul_ortalama["time"].iloc[-1])

# Efsaneyi gösterme
plt.legend()

# Grafikleri gösterme
plt.tight_layout()
plt.show()

In [ ]:
!pip install xlrd

## Emisyon ve Sıcaklık

In [ ]:
# türkiye gaz emisyonlarını görsele dahil ediyorum

emisyon_tuik = pd.read_excel("sera gazi emisyonlari (milyon ton co2 esdegeri).xls")
new_column_names = {
    "Sera gazı emisyonları (CO2 eşdeğeri), 1990 - 2021": "Yıl",
    "Unnamed: 1": "Toplam",
    "Unnamed: 2": "CO2",
    "Unnamed: 3": "CH4",
    "Unnamed: 4": "N2O",
    "Unnamed: 5": "FGAZ"
}

emisyon_tuik.rename(columns=new_column_names, inplace=True)
emisyon_tuik = emisyon_tuik.iloc[3:35]
emisyon_tuik['Yıl'] = pd.to_datetime(emisyon_tuik['Yıl'], format='%Y')
emisyon_tuik.reset_index(drop=True, inplace=True)

In [ ]:
emisyon_tuik

In [ ]:
# Grafiğin boyutunu ayarlayın
plt.figure(figsize=(12, 8))

# Arka planı beyaz yerine grid tarzı yapın
sns.set(style="whitegrid")

# Birinci y-eksen verileri çizdirme (sıcaklık)
plt.plot(istanbul_ortalama["time"], ten_year_avg, label="10 Yıllık Ortalama Sıcaklık", linewidth=2, color="r", marker='o')

# Birinci y-eksen etiketlerini ve başlığı güncelleme
plt.xlabel("Tarih")
plt.ylabel("Sıcaklık Ortalaması (°C)")
plt.title("İstanbul 10 Yıllık Ortalama Sıcaklık ve Türkiye Emisyon Oranı TÜİK")
plt.legend()

# İlgili yıla odaklanmak için grafiği sınırlama
plt.xlim(istanbul_ortalama["time"].iloc[0], istanbul_ortalama["time"].iloc[-1])

# İkinci y-eksen verileri çizdirme (emisyon oranları)
ax2 = plt.gca().twinx()  # İkincil y-eksen oluşturma
ax2.plot(emisyon_tuik["Yıl"], emisyon_tuik["Toplam"], label="Emisyon Oranı TÜİK", linewidth=2, color="orange", marker='x')
ax2.set_ylabel("Emisyon Oranı Türkiye")  # İkincil y-eksen etiketi

plt.legend(bbox_to_anchor=(0.2, 0.9555))

# Grafikleri gösterme
plt.tight_layout()
plt.show()

In [ ]:
#CAMBRIDGE VE OXFORD DESTEKLI OWID DATASINA GÖRE TR EMİSYON ORANLARINA BAKALIM
emisyon_owid = pd.read_csv("owid-co2-data.csv")

In [ ]:
emisyon_owid.loc[emisyon_owid.loc[emisyon_owid.country == "Turkey"]["co2"].dropna().index].columns
emisyon_owid = emisyon_owid.loc[emisyon_owid.loc[emisyon_owid.country == "Turkey"]["co2"].dropna().index][["country","year","co2","gas_co2","methane","nitrous_oxide","oil_co2",'other_industry_co2', 'primary_energy_consumption',"consumption_co2","energy_per_capita",'total_ghg']]
emisyon_owid = emisyon_owid.fillna(0)
emisyon_owid["total"] = emisyon_owid["co2"] + emisyon_owid["methane"] + emisyon_owid["nitrous_oxide"]
emisyon_owid_tur = emisyon_owid.copy()
emisyon_owid_tur = emisyon_owid_tur[["country","year","total_ghg","total"]]
emisyon_owid_tur.loc[emisyon_owid_tur["total_ghg"]==0, "total_ghg"] = emisyon_owid_tur["total"]
emisyon_owid_tur.drop(columns="total", inplace=True)
emisyon_owid_tur['year'] = pd.to_datetime(emisyon_owid_tur['year'], format='%Y')

In [ ]:
emisyon_owid_tur

In [ ]:
# Grafiğin boyutunu ayarlayın
plt.figure(figsize=(12, 8))

# Arka planı beyaz yerine grid tarzı yapın
sns.set(style="whitegrid")

# Birinci y-eksen verileri çizdirme (sıcaklık)
plt.plot(istanbul_ortalama["time"], ten_year_avg, label="10 Yıllık Ortalama Sıcaklık", linewidth=2, color="r", marker='o')

# Birinci y-eksen etiketlerini ve başlığı güncelleme
plt.xlabel("Tarih")
plt.ylabel("Sıcaklık Ortalaması (°C)")
plt.title("İstanbul 10 Yıllık Ortalama Sıcaklık ve Türkiye Emisyon Oranı TÜİK ve OWID")
plt.legend()

# İlgili yıla odaklanmak için grafiği sınırlama
plt.xlim(istanbul_ortalama["time"].iloc[0], istanbul_ortalama["time"].iloc[-1])

# İkinci y-eksen verileri çizdirme (emisyon oranları)
ax2 = plt.gca().twinx()  # İkincil y-eksen oluşturma
ax2.plot(emisyon_tuik["Yıl"], emisyon_tuik["Toplam"], label="Emisyon Oranı TÜİK", linewidth=2, color="orange", marker='x')
ax2.set_ylabel("Emisyon Oranı Türkiye TÜİK")  # İkincil y-eksen etiketi
plt.legend(bbox_to_anchor=(0.2, 0.9555))

# Üçüncü y-eksen verileri çizdirme (emisyon oranları)
ax3 = plt.gca().twinx()  
ax3.plot(emisyon_owid_tur[emisyon_owid_tur["year"] >= datetime(1930, 1, 1)]["year"], emisyon_owid_tur[emisyon_owid_tur["year"] >= datetime(1930, 1, 1)]["total_ghg"], label="Emisyon Oranı OWID", linewidth=2, color="blue", marker='x')
ax3.set_ylabel("Emisyon Oranı Türkiye OWID") 
plt.legend(bbox_to_anchor=(0.205, 0.91))


# Grafikleri gösterme
plt.tight_layout()
plt.show()

## Yeşillik ve Sıcaklık

In [ ]:
yesillik_istanbul = pd.read_excel("istanbul data/2022-yeil-alan-bilgileri.xlsx")
yesillik_istanbul.drop(columns="Unnamed: 20", inplace=True)

In [ ]:
dikilen_agac_istanbul = pd.melt(yesillik_istanbul.loc[yesillik_istanbul["FAALİYET KONUSU"]=="Dikilen Ağaç Sayısı"].drop(columns="BİRİM"), id_vars=["FAALİYET KONUSU"], var_name="Year", value_name="Count")
bakim_agac_istanbul = pd.melt(yesillik_istanbul.loc[yesillik_istanbul["FAALİYET KONUSU"]=="Bakımı Yapılan Ağaç Sayısı"].drop(columns="BİRİM"), id_vars=["FAALİYET KONUSU"], var_name="Year", value_name="Count")
dikilen_cali_istanbul = pd.melt(yesillik_istanbul.loc[yesillik_istanbul["FAALİYET KONUSU"]=="Dikilen Çalı Sayısı"].drop(columns="BİRİM"), id_vars=["FAALİYET KONUSU"], var_name="Year", value_name="Count")

In [ ]:
dikilen_agac_istanbul['year'] = pd.to_datetime(dikilen_agac_istanbul['Year'], format='%Y')
bakim_agac_istanbul['year'] = pd.to_datetime(bakim_agac_istanbul['Year'], format='%Y')
dikilen_cali_istanbul['year'] = pd.to_datetime(dikilen_cali_istanbul['Year'], format='%Y')

In [ ]:
dikilen_agac_istanbul

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Grafiğin boyutunu ayarlayın
plt.figure(figsize=(12, 8))

# Arka planı beyaz yerine grid tarzı yapın
sns.set_theme(style='white')

# Birinci y-eksen verileri çizdirme (sıcaklık)
plt.plot(dikilen_cali_istanbul["year"], dikilen_cali_istanbul["Count"], label="İstanbul Dikilen Çalı Sayısı", linewidth=2, color="orange", marker='o')
plt.legend(loc='center left')

# İkinci y-eksen verileri çizdirme (emisyon oranları)
ax2 = plt.gca().twinx()  
ax2.plot(bakim_agac_istanbul["year"], bakim_agac_istanbul["Count"], label="İstanbul Bakımı Yapılan Ağaç Sayısı", linewidth=2, color="purple", marker='x')

# Üçüncü y-eksen verileri çizdirme (Dikilen Ağaç Sayısı)
ax3 = plt.gca().twinx()  # İkincil y-eksen oluşturma
ax3.plot(dikilen_agac_istanbul["year"], dikilen_agac_istanbul["Count"], label="İstanbul Dikilen Ağaç Sayısı", linewidth=2, color="green", marker='x')

# Tüm y-eksen etiketlerini güncelleme
plt.ylabel("Değerler")

# Başlığı güncelleme
plt.title("Çeşitli Verilerin İstanbul Üzerindeki Etkisi")

# Tüm çizgi açıklamalarını ekleme
lines = plt.gca().get_lines() + ax2.get_lines()
labels = [line.get_label() for line in lines]
plt.legend(lines, labels, loc='upper left')

# Grafikleri gösterme
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Grafiğin boyutunu ayarlayın
plt.figure(figsize=(12, 8))

# Arka planı beyaz yerine grid tarzı yapın
sns.set_theme(style='white')

# Birinci y-eksen verileri çizdirme (sıcaklık)
plt.plot(istanbul_ortalama["time"], ten_year_avg, label="10 Yıllık Ortalama Sıcaklık", linewidth=2, color="r", marker='o')
plt.legend(loc='center left')

# İkinci y-eksen verileri çizdirme (emisyon oranları)
ax2 = plt.gca().twinx()  
ax2.plot(emisyon_owid_tur[emisyon_owid_tur["year"] >= datetime(1930, 1, 1)]["year"], emisyon_owid_tur[emisyon_owid_tur["year"] >= datetime(1930, 1, 1)]["total_ghg"], label="Emisyon Oranı OWID", linewidth=2, color="blue", marker='x')

# Üçüncü y-eksen verileri çizdirme (Dikilen Ağaç Sayısı)
ax3 = plt.gca().twinx()  # İkincil y-eksen oluşturma
ax3.plot(dikilen_agac_istanbul["year"], dikilen_agac_istanbul["Count"], label="İstanbul Dikilen Ağaç Sayısı", linewidth=2, color="green", marker='x')

# Tüm y-eksen etiketlerini güncelleme
plt.ylabel("Değerler")

# Başlığı güncelleme
plt.title("Çeşitli Verilerin İstanbul Üzerindeki Etkisi")

# Tüm çizgi açıklamalarını ekleme
lines = plt.gca().get_lines() + ax2.get_lines()
labels = [line.get_label() for line in lines]
plt.legend(lines, labels, loc='upper left')

# Grafikleri gösterme
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Grafiğin boyutunu ayarlayın
plt.figure(figsize=(12, 8))

# Arka planı beyaz yerine grid tarzı yapın
sns.set_theme(style='white')

# Birinci y-eksen verileri çizdirme (sıcaklık)
plt.plot(istanbul_ortalama["time"], ten_year_avg, label="10 Yıllık Ortalama Sıcaklık", linewidth=2, color="r", marker='o')
plt.legend(loc='center left')

# İkinci y-eksen verileri çizdirme (emisyon oranları)
ax2 = plt.gca().twinx()  
ax2.plot(emisyon_owid_tur[emisyon_owid_tur["year"] >= datetime(1930, 1, 1)]["year"], emisyon_owid_tur[emisyon_owid_tur["year"] >= datetime(1930, 1, 1)]["total_ghg"], label="Emisyon Oranı OWID", linewidth=2, color="blue", marker='x')

# Üçüncü y-eksen verileri çizdirme (bakımı yapılan ağaç sayısı)
ax3 = plt.gca().twinx()
ax3.plot(bakim_agac_istanbul["year"], bakim_agac_istanbul["Count"], label="İstanbul Bakımı Yapılan Ağaç Sayısı", linewidth=2, color="purple", marker='s')

# Tüm y-eksen etiketlerini güncelleme
plt.ylabel("Değerler")

# Başlığı güncelleme
plt.title("Çeşitli Verilerin İstanbul Üzerindeki Etkisi")

# Tüm çizgi açıklamalarını ekleme
lines = plt.gca().get_lines() + ax2.get_lines()
labels = [line.get_label() for line in lines]
plt.legend(lines, labels, loc='upper left')

# Grafikleri gösterme
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Grafiğin boyutunu ayarlayın
plt.figure(figsize=(12, 8))

# Arka planı beyaz yerine grid tarzı yapın
sns.set_theme(style='white')

# Birinci y-eksen verileri çizdirme (sıcaklık)
plt.plot(istanbul_ortalama["time"], ten_year_avg, label="10 Yıllık Ortalama Sıcaklık", linewidth=2, color="r", marker='o')
plt.legend(loc='center left')

# İkinci y-eksen verileri çizdirme (emisyon oranları)
ax2 = plt.gca().twinx()  
ax2.plot(emisyon_owid_tur[emisyon_owid_tur["year"] >= datetime(1930, 1, 1)]["year"], emisyon_owid_tur[emisyon_owid_tur["year"] >= datetime(1930, 1, 1)]["total_ghg"], label="Emisyon Oranı OWID", linewidth=2, color="blue", marker='x')

# Üçüncü y-eksen verileri çizdirme (dikilen çalı sayısı)
ax3 = plt.gca().twinx()
ax3.plot(dikilen_cali_istanbul["year"], dikilen_cali_istanbul["Count"], label="İstanbul Dikilen Çalı Sayısı", linewidth=2, color="orange", marker='d')

# Tüm y-eksen etiketlerini güncelleme
plt.ylabel("Değerler")

# Başlığı güncelleme
plt.title("Çeşitli Verilerin İstanbul Üzerindeki Etkisi")

# Tüm çizgi açıklamalarını ekleme
lines = plt.gca().get_lines() + ax2.get_lines()
labels = [line.get_label() for line in lines]
plt.legend(lines, labels, loc='upper left')

# Grafikleri gösterme
plt.tight_layout()
plt.show()


## Su ve Sıcaklık

In [ ]:
yagis_baraj = pd.read_excel("istanbul data/ibb_yags_verileri_2011-23.xlsx")
yagis_baraj['year'] = pd.to_datetime(yagis_baraj['Tarih'], format='%Y')

In [ ]:
yagis_baraj

In [ ]:
# Tarih sütununu pandas'ın datetime formatına çevirin
yagis_baraj['year'] = pd.to_datetime(yagis_baraj['year'])

# Tarihe göre gruplayıp aylık ortalama değerleri hesaplayın
yagis_baraj['month'] = yagis_baraj['year'].dt.month
yagis_baraj['year'] = yagis_baraj['year'].dt.year

In [ ]:
yagis_baraj

In [ ]:
# istanbul ortalama'daki aynı formatta year sütunu ekliyorum
yagis_baraj['yearS'] = pd.to_datetime(yagis_baraj['year'].astype(str) + '-12-31')

In [ ]:
yagis_yillik_istanbul = yagis_baraj.groupby("year").sum().drop(columns="month")
yagis_yillik_istanbul.reset_index(drop=False, inplace=True)
yagis_yillik_istanbul

In [ ]:
# istanbul ortalama'daki aynı formatta year sütunu ekliyorum
yagis_yillik_istanbul['yearS'] = pd.to_datetime(yagis_yillik_istanbul['year'].astype(str) + '-12-31')

In [ ]:
yagis_yillik_istanbul["TotalBaraj"]=yagis_yillik_istanbul["Ömerli"]+yagis_yillik_istanbul["Darlik"]+yagis_yillik_istanbul["Elmali"]+yagis_yillik_istanbul["Terkos"]+yagis_yillik_istanbul["B.çekmece"]+yagis_yillik_istanbul["Sazlidere"]+yagis_yillik_istanbul["Alibey"]+yagis_yillik_istanbul["Kazandere"]+yagis_yillik_istanbul["Pabuçdere"]
yagis_yillik_istanbul

In [ ]:
istanbul_ortalama.loc[istanbul_ortalama["time"] >= pd.to_datetime('2010-01-01')]

In [ ]:
su_tuketim_m3 = pd.read_excel("istanbul data/m3-in-total-since-1991.xlsx")
su_tuketim_yesil = pd.read_excel("istanbul data/yllara-gore-yeil-alanlar-icin-harcanan-su-miktar.xlsx")

In [ ]:
su_tuketim_yesil

In [ ]:
su_tuketim_m3

In [ ]:
baraj_doluluk_istanbul = pd.read_excel("istanbul data/istanbul-dams-daily-occupancy-rates.xlsx")
#totali toplayıp eleman sayısına bölüyorum
baraj_doluluk_istanbul["TotalBaraj"]=baraj_doluluk_istanbul["Ömerli"]+baraj_doluluk_istanbul["Istrancalar"]+baraj_doluluk_istanbul["Darlık"]+baraj_doluluk_istanbul["Elmalı"]+baraj_doluluk_istanbul["Terkos"]+baraj_doluluk_istanbul["Büyükçekmece"]+baraj_doluluk_istanbul["Sazlıdere"]+baraj_doluluk_istanbul["Alibey"]+baraj_doluluk_istanbul["Kazandere"]+baraj_doluluk_istanbul["Pabuçdere"]
baraj_doluluk_istanbul["TotalBaraj"]=baraj_doluluk_istanbul["TotalBaraj"]/10
#yıllık ortalamaları buluyorum
baraj_doluluk_istanbul["Tarih"] = pd.to_datetime(baraj_doluluk_istanbul["Tarih"])  # Convert "Tarih" column to datetime
baraj_doluluk_istanbul["year"] = baraj_doluluk_istanbul["Tarih"].dt.year  # Extract year from the datetime column
yillik_ort = baraj_doluluk_istanbul.groupby("year")["TotalBaraj"].mean()
baraj_doluluk_istanbul["yillik_ort"] = baraj_doluluk_istanbul["year"].map(yillik_ort)

baraj_doluluk_istanbul

In [ ]:
plt.figure(figsize=(10, 6))  # Grafik boyutunu belirleme
plt.bar(yagis_yillik_istanbul["year"], yagis_yillik_istanbul["TotalBaraj"], color="lightblue")
plt.xlabel("Yıl")
plt.ylabel("Yağış Miktarı (mm)")
plt.title("Toplam Baraj Yağışları İstanbul")
plt.xticks(rotation=45)  # Eksen etiketlerini 45 derece döndürme
plt.grid(axis="y", linestyle="--", alpha=0.7)  # Y ekseni üzerinde ızgaralar ekler
plt.tight_layout()  # Grafik öğelerinin sığdırılması
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))  # Grafik boyutunu belirleme
plt.bar(su_tuketim_m3["YIL"], su_tuketim_m3["TOPLAM M3"], color="lightblue")
plt.xlabel("Yıl")
plt.ylabel("Tüketim Miktarı (m3)")
plt.title("Su Tüketim Miktarı Yıllık")
plt.xticks(rotation=45)  # Eksen etiketlerini 45 derece döndürme
plt.grid(axis="y", linestyle="--", alpha=0.7)  # Y ekseni üzerinde ızgaralar ekler
plt.tight_layout()  # Grafik öğelerinin sığdırılması
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))  # Grafik boyutunu belirleme
plt.bar(su_tuketim_yesil["Yıl"], su_tuketim_yesil["Su Tüketimi (m3)"], color="lightblue")
plt.xlabel("Yıl")
plt.ylabel("Tüketim Miktarı (m3)")
plt.title("Yeşil Alan Su Tüketim Miktarı Yıllık")
plt.xticks(rotation=45)  # Eksen etiketlerini 45 derece döndürme
plt.grid(axis="y", linestyle="--", alpha=0.7)  # Y ekseni üzerinde ızgaralar ekler
plt.tight_layout()  # Grafik öğelerinin sığdırılması
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))  # Grafik boyutunu belirleme
plt.bar(baraj_doluluk_istanbul["year"], baraj_doluluk_istanbul["yillik_ort"], color="lightblue")
plt.xlabel("Yıl")
plt.ylabel("Yıllık ortalama baraj doluluk oranı")
plt.title("Ortalama Baraj Doluluk Oranı İstanbul")
plt.xticks(rotation=45)  # Eksen etiketlerini 45 derece döndürme
plt.grid(axis="y", linestyle="--", alpha=0.7)  # Y ekseni üzerinde ızgaralar ekler
plt.tight_layout()  # Grafik öğelerinin sığdırılması
plt.show()

## Trafik ve Sıcaklık

In [ ]:
yagis_baraj